<div style="background:#E9FFF6; color:#440404; padding:8px; border-radius: 4px; text-align: center; font-weight: 500;">IFN619 - Data Analytics for Strategic Decision Makers (2023_sem1)</div>

# IFN619 :: C4 - Machine Learning

Machine Learning is a process of data analysis where the computer 'learns' a model of a particular aspect of the data being analysed. The process involves 'training' whereby a particular type of model is 'fit' to the data, creating a model which can be used to classify or predict from within the same data, or on new data. 

The success of machine learning typically relies on selecting an appropriate ML approach that suits both the data and the question being answered. For example, the topic modelling that we did in C1 involved the assumption that there were latent topics in the data that could be analysed by fitting a topic model to the data. If this assumption was true, if the data was appropriately structured, and if we selected a reasonable value for 'k' (number of topics), then we were able to obtain helpful topics that described that underlying data. However, if any of these conditions were not met, then the quality of the results deteriorated. Thus, when using ML, it is important to know your data, understand what question/s you are trying to address, know which ML models might be appropriate, and be willing to experiment with different combinations of models and parameters in order to obtain reliable results.

To help select an approach, we can answer 2 questions:

1. What kind of data?
    - LABELLED DATA -> Supervised ML
    - UNLABELLED DATA => Unsupervised ML

2. What do we want to achieve?
    - PREDICT -> Regression algorithms
    - CLASSIFY -> Clustering algorithms

*Note: The above don't cover all ML, but provide a good starting point for exploration*

In this session we are going to look at 3 simple examples:

1. Classify unlabelled data into clusters (groups) using a K-means algorithm
2. Predict a value for new data based on existing labelled data using Linear Regression
3. Predict a label for new data based on existing labelled data using Logistic Regression

In [ ]:
from sklearn.preprocessing import minmax_scale

from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import pandas as pd
import plotly.express as px


### K-means algorithm

K-means is an iterative algorithm that aims to identify centroids (means) in the data for a given number of clusters (k). It then assigns a cluster label (K-value) to each observation in the data based on the nearest centroid (mean). For more details see this [Wikipedia page](https://en.wikipedia.org/wiki/K-means_clustering)

![k-means.pdf](./graphics/k-means-wikipedia.pdf)

[Image from Wikipedia](https://en.wikipedia.org/wiki/K-means_clustering)

### EXAMPLE - Fair work place

As a starting example, we will use the 'toy' fair work place data that we explored in the ethics module. 

### Data preparation

The first thing we notice is that the data is not all numeric (workers are W, male is M, etc), and that the numerical data is on an arbitrary scale. To ensure that we can cluster using Euclidean distance it is important to normalise the components (put them on the same scale) which typically is 0-1. This means scaling the rating to be between 0 and 1, and converting the text to numbers.

In [ ]:
# Load the data

file_path = "data/"
file_name = "fair-workplace-survey.csv"
fwp_df = pd.read_csv(f"{file_path}{file_name}", index_col="id")
fwp_df

In [ ]:
# Convert Text to numbers

conversions = {"W":1,"S":0,"F":1,"M":0,"O":0.5} # Note that allowing an 'other' value can accommodate future data

# Get numeric values for Role
fwp_df["Role_n"] = fwp_df['Role'].apply(lambda x: conversions[x])
fwp_df


In [ ]:
# Get numeric values for gender

fwp_df["Gender_n"] = fwp_df['Gender'].apply(lambda x: conversions[x])
fwp_df

In [ ]:
# Scale the fair workplace rating data

minmax_scale(fwp_df['FairWorkPlace'])

In [ ]:
fwp_df['rating_n'] = minmax_scale(fwp_df['FairWorkPlace'])
fwp_df

#### Clustering

Now we have numerical data with all components on the same scale, we can perform the clustering.

To do so, we need to set the number of clusters we want as these are the number of groups that the algorithm is going to try and fit the data to.

In [ ]:
# K-means - 2 clusters
km2 = KMeans(n_clusters=2, random_state=0).fit(fwp_df[['Role_n','Gender_n','rating_n']])
fwp_df['km2'] = km2.labels_
fwp_df

In [ ]:
# K-means - 3 clusters
km3 = KMeans(n_clusters=3, random_state=0).fit(fwp_df[['Role_n','Gender_n','rating_n']])
fwp_df['km3'] = km3.labels_
fwp_df

In [ ]:
# K-means - 4 clusters
km4 = KMeans(n_clusters=4, random_state=0).fit(fwp_df[['Role_n','Gender_n','rating_n']])
fwp_df['km4'] = km4.labels_
fwp_df

#### Visualise the clusters

We can now visualise the clusters - this would be particularly helpful if the dataset was large.

In [ ]:
fig = px.scatter_3d(fwp_df, x='Role_n', y='Gender_n', z='rating_n',
              color='km4')
fig.show()

### Supervised ML - Training and Testing

Supervised machine learning algorithms use labeled datasets to predict labels (categorical data) or values (numerical data). As input data is fed into the model, it adjusts its weights until the model has been fitted appropriately. Supervised machine learning algorithms help organizations solve for a variety of real-world problems at scale, such as predicting future income or if a person is going to default a credit.

Supervised machine learning uses a portion of the data (training set) to train a given model to yield the given labels/values. The remainder of the data (test set) can be used to test the effectiveness of the model against the provided labels/values. Once the model is yielding reasonable results, it can be used to predict labels/values for unseen data (data that hasn't been labelled).

- The training dataset includes inputs and correct outputs, which allow the model to learn over time 
- The testing dataset includes inputs and correct outputs but the model is only given the inputs. The model is evaluated by comparing its predicted value (model outputs) to the actual value (correct outputs)


### Linear Regression algorithm

Linear regression is used to identify the relationship between a dependent variable and one or more independent variables and is typically leveraged to make predictions about future outcomes. The results from the linear regression help in predicting an unknown value depending on the relationship with the predicting variables. Linear regression fits a straight line that minimizes the discrepancies between predicted and actual output values. Further information in [Linear regression](https://en.wikipedia.org/wiki/Linear_regression).

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Linear_least_squares_example2.svg/440px-Linear_least_squares_example2.svg.png" style="width:50%">

#### Data

Load and get to know the data.

The included dataset is derived from a [dataset of realestate property prices in Darwin](https://www.kaggle.com/datasets/thedevastator/australian-housing-data-1000-properties-sampled?resource=download) 

In [ ]:
file_path = "data/"
file_name = "RealEstateAU_NT_property.csv"
property_df = pd.read_csv(f"{file_path}{file_name}")
property_df

In [ ]:
# Get the descriptive statiistics
property_df.describe()

Are their any obvious correlations in the data?

In [ ]:

property_corr = property_df.corr()
property_corr

In [ ]:
# Create a heatmap to visualise the correlations
pc_fig = px.imshow(property_corr) 
pc_fig.show()

Parking is least correlated with price, so we can leave this out of the model and focus on those variables that correlate most with what we're interested in. Further for this model, we're only interested in numerical data, so we can leave out the property variable also (which is categorical).

In [ ]:
price_df = property_df[['price','bedrooms','bathrooms']]
price_df

Different visualisations can sometimes help better understand the relationships in the data. When looking for linear relationships, a pair plot can be helpful...

In [ ]:
price_fig = px.scatter_matrix(price_df) # Create a pair plot to see the linearity of the variables
price_fig.show()

To fit a linear regression model, we need to assign the dependent variable that we want to predict to the Y-axis, and draw the X-axis data from the dependent variables.

In [ ]:
# Independent variables

X_data = price_df[['bedrooms','bathrooms']]
X_data

In [ ]:
# Dependent variable

y_data = price_df['price']
y_data

In [ ]:
# Break the current dataset into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, shuffle=True, train_size=0.8, random_state=99) # Train size determines the perceptage use for training the model

In [ ]:
# Create a new linear regression model
linear_model = LinearRegression() 

# Train the model with the train dataset
linear_model.fit(X_train, y_train) 

# Predict using the testing dataset
linear_predictions = linear_model.predict(X_test) 
linear_predictions

In [ ]:
# Calculate the error between the testing dataset the model's predictions
linear_R2 = r2_score(y_test, linear_predictions) 

print(f'The model R squared score is: {linear_R2}')

The **R-squared** is a coeficient between 0 and 1 that determine the quality of the model prediction. This number indicates the percentage of variance in the dependent variable that the independent variables explain. 0 means that the model's prediction is not explained at all by the independent variables, while 1 means that the model's prediction is 100% explained by the independent variables.

In [ ]:
# Create a chart to check the differences between what has been predicted and the real values
y_test_fig_df = pd.DataFrame(y_test)
linear_prediction_fig_df = pd.DataFrame(linear_predictions)
linear_prediction_fig_df.columns = ['Predicted Price']
linear_prediction_fig_df['Test Index'] = y_test_fig_df.index
linear_prediction_fig_df.set_index('Test Index', inplace=True)
linear_fig_df = linear_prediction_fig_df.join(y_test_fig_df)
linear_fig = px.scatter(linear_fig_df)
linear_fig.show()


### Logistic Regression algorithm

While linear regression is leveraged when dependent variables are continuous quantitative data, logistic regression can be used when the dependent variable is categorical. The most common type of logistic regression works with dependent variables with binary values such as "true" and "false" or "yes" and "no." While both regression models seek to understand relationships between data inputs, logistic regression is mainly used to solve binary classification problems. Further information in [Logistic regression](https://en.wikipedia.org/wiki/Logistic_regression).

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Exam_pass_logistic_curve.svg/800px-Exam_pass_logistic_curve.svg.png" style="width: 50%">

#### Data

For this example, we will use the same data as before, except instead of predicting price, we will see if we can predict as 'house' or 'apartment'. While this might not appear to be useful (as we would usually know what a property is), by predicting the relationships, we can identify anomalies. e.g. data that is different than what is expected. This might help with tasks such as setting an appropriate price for sale.

First it is important to understand the balance between values of the dependent variable

In [ ]:
property_df.groupby('property').count()

These are reasonably balanced so it should not be a problem in the analysis. Skewed data can be problematic as there may not be sufficient data for one dependent value to train an effective model.

As with the linear regression, we need to create separate datasets for dependent, independent, and training and test.

In [ ]:
# Independent variables data - start with all options
X_data = property_df[['price','bedrooms','bathrooms','parking']] 

# Dependent variable data
y_data = property_df['property']=='House' # convert to True False 

In [ ]:
y_data

In [ ]:
# Break the current dataset into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, shuffle=True, train_size=0.8, random_state=99) # Train size determines the perceptage use for training the model

In [ ]:
# Check the class balance
y_train.value_counts(normalize=True)

There is a small class inbalance in the variable that we are going to predict. Therefore, the model is likely to predict towards 'No' just because the biased data rather than the independent variables. In any classification model such as logistic regression, decision trees, etc. The class balance need to be considered.

In [ ]:
logistic_model = LogisticRegression(class_weight={False: 0.53, True: 0.47})

Additionally, it is a common practice to scale the date to have a better model. To scale the data wwe are going to use standardization that scale the data to have a mean of 0 and a standard deviation of 1.

In [ ]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [ ]:
# Fit the model to the training dataset
logistic_model.fit(X_train, y_train)

To evaluate the model the most common method is the confusion matrix.

In [ ]:
logistic_prediction = logistic_model.predict(X_test) # Use the model to predict based on the testing dataset
cm = confusion_matrix(y_test, logistic_prediction) # Compare the model's prediction against the true value in the testing dataset
cm

In [ ]:
cm_fig = px.imshow(cm, labels={'x': 'Predicted label', 'y': 'Actual label'})
cm_fig.show()

In [ ]:
report = classification_report(y_test, logistic_prediction) # Get further evaluation metrics
print(report)

#### Precision and Recall

Precision and recall are important factors in understanding the effectiveness of a model.

- Precision: What proportion of positive identifications was actually correct? 
- Recall: What proportion of actual positives was identified correctly?

See this[Google developers page](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall) for a helpful explanation.
See this [Wikipedia page](https://en.wikipedia.org/wiki/Precision_and_recall) for more information.


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/700px-Precisionrecall.svg.png" width="30%">

[Image source](https://commons.wikimedia.org/wiki/File:Precisionrecall.svg)

### Further exploration

Take a look at other ML approaches available using the [Scikit-learn library](https://scikit-learn.org/stable/).

Try different approaches with your own data.